# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f890fbf6490>
├── 'a' --> tensor([[ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506]])
└── 'x' --> <FastTreeValue 0x7f890fbf6130>
    └── 'c' --> tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                        [ 0.6601, -1.3868,  0.0134, -1.5892],
                        [-0.7075, -0.2579, -0.3554, -0.1606]])

In [4]:
t.a

tensor([[ 0.6298, -0.1541,  0.8906],
        [-0.9172, -1.5432, -1.3506]])

In [5]:
%timeit t.a

61.6 ns ± 0.0643 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f890fbf6490>
├── 'a' --> tensor([[-0.9546, -0.8433,  0.6158],
│                   [ 0.1036,  0.9166,  0.6188]])
└── 'x' --> <FastTreeValue 0x7f890fbf6130>
    └── 'c' --> tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                        [ 0.6601, -1.3868,  0.0134, -1.5892],
                        [-0.7075, -0.2579, -0.3554, -0.1606]])

In [7]:
%timeit t.a = new_value

66.9 ns ± 0.0529 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506]]),
    x: Batch(
           c: tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                      [ 0.6601, -1.3868,  0.0134, -1.5892],
                      [-0.7075, -0.2579, -0.3554, -0.1606]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6298, -0.1541,  0.8906],
        [-0.9172, -1.5432, -1.3506]])

In [11]:
%timeit b.a

57.4 ns ± 0.0557 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5669, -0.0292, -0.3897],
               [-2.6579,  0.0777, -1.0719]]),
    x: Batch(
           c: tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                      [ 0.6601, -1.3868,  0.0134, -1.5892],
                      [-0.7075, -0.2579, -0.3554, -0.1606]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.241 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.0742 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 27.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f88e40c4cd0>
├── 'a' --> tensor([[[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]],
│           
│                   [[ 0.6298, -0.1541,  0.8906],
│                    [-0.9172, -1.5432, -1.3506]]])
└── 'x' --> <FastTreeValue 0x7f88e40d6c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 73.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f890cb84f70>
├── 'a' --> tensor([[ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506],
│                   [ 0.6298, -0.1541,  0.8906],
│                   [-0.9172, -1.5432, -1.3506]])
└── 'x' --> <FastTreeValue 0x7f890cb84820>
    └── 'c' --> tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                        [ 0.6601, -1.3868,  0.0134, -1.5892],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 87.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.9 µs ± 82.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]],
       
               [[ 0.6298, -0.1541,  0.8906],
                [-0.9172, -1.5432, -1.3506]]]),
    x: Batch(
           c: tensor([[[ 0.4839,  0.4579,  0.4785,  0.1805],
                       [ 0.6601, -1.3868,  0.0134, -1.5892],
                       [-0.7075, -0.2579, -0.3554, -0.1606]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 95.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506],
               [ 0.6298, -0.1541,  0.8906],
               [-0.9172, -1.5432, -1.3506]]),
    x: Batch(
           c: tensor([[ 0.4839,  0.4579,  0.4785,  0.1805],
                      [ 0.6601, -1.3868,  0.0134, -1.5892],
                      [-0.7075, -0.2579, -0.3554, -0.1606],
                      [ 0.4839,  0.4579,  0.4785,  0.1805],
                      [ 0.6601, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 437 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

301 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
